In [48]:
# Probability distribution functions are "mathematical models", which means that
# they are abstract descriptions of a concrete system, and as such, we 
# can understand the behavior of such systems, experiment with them, modify them,
# and predict them. We can fuck them

# Mathematical models are not just about probability distributions, they also can
# be functions, a sum, a railroad diagram, a matrix. Anything that is a representation
# of a concrete system using math language.

# Programming allows us to run simulations starting from those mathematical 
# abstractions. In one word, excecution. Programming lets us navigate from abstraction
# To the real of the plausible and experimentable.

# Simpy is a library designed for simulating discrete-event simulations. 
# (simulations with concrete objects and processes). And it is a process-based
# framework, which means that it allows someone to simulate systems in which
# concrete objects participate in concrete processes.  

import simpy

# As an example. 
    # Imagine you own a second-hand car store. 
    # You have just opened, so you don’t have a very large volume 
    # of work yet, but you already have some initial data. 
    # You have found that during a week, as a general rule, you 
    # receive 5 cars from private individuals. You have also seen
    # that the volume of customers is higher, with 20 potential customers 
    # coming in a week as ageneral rule.
# If I wished to model and simulate this car store, It will be better to have a
# mathematical description (an abstraction of this car store). I can say that:

import numpy as np # We need this library for better manipulating numbers 

# We could use a poisson distribution in order to model the customers arriving
# A poisson distribution is a probability distribution that models the occurence
# of rare events happening in a fixed intervavl of time.
# The poisson distribution uses the parameter lambda = the expected value in the interval.

# Define parameters of the poisson distribution
lambda_cars = 5  # Mean number of cars received per week
lambda_customers = 20  # Mean number of potential customers per week
p_purchase = 0.2  # Probability of a potential customer purchasing a car

# Set a random seed for this simulation, a random seed is 
np.random.seed(42)

# Simulate number of cars received and potential customers
num_cars_received = np.random.poisson(lambda_cars) # Generate a random number from the poisson distribution with lambda = 5
num_potential_customers = np.random.poisson(lambda_customers) # Generate a random number from the poisson distribution with lambda = 20

# Calculate expected number of cars sold 
num_cars_sold = min(num_cars_received, int(num_potential_customers * p_purchase))

print(f"Expected number of cars sold: {num_cars_sold}")



Expected number of cars sold: 4


In [196]:
# In simulations, we might encounter ourselves simulating a lot of things at once, in 
# order to not use a lot of computational resources, we can use generators, which are
# basically functions whose outputs are not stored in memory. 

# I can imagine a generator as an abstract sphere which spins static in the air, and each time I call it it
# pulses something, the generator changes color as it funciton changes, and can pulse thigs at a different speed
# Here is a generator of lines of a file

def fileLines(fileName):
    with open(fileName) as file:
        for line in file:
            yield line

# And here is how I can use that generator
# (Defining a function is not actually needed but I'll do it)

def printFile(fileName):
    for line in fileLines(fileName):
        return line

printFile("Hola.txt")

'Hola\n'

In [197]:
# Now I could do the same with a function

def printFile2(fileName):
    with open(fileName) as file:
        for line in file:
            return line

printFile2("Hola.txt")

# However, the function that uses the generator does so in less time
# Than the function that doesn't

import time

def generatorRuntime():
    start = time.time()
    printFile("Hola.txt")
    end = time.time()
    return end - start

def functionRuntime():
    start = time.time()
    printFile2("Hola.txt")
    end = time.time()
    return end - start

print(f"Time taken by function with generator: {generatorRuntime():.50f} ")
print(f"Time taken by function with generator: {functionRuntime():.50f} ")


Time taken by function with generator: 0.00000000000000000000000000000000000000000000000000 
Time taken by function with generator: 0.00000000000000000000000000000000000000000000000000 


In [241]:
# It seems that the time is not constant, I will collect a sample for each time
# and compare each mean

def timeSampleGenMean():
    sample = [] 
    for i in range(1000):
        sample.append(generatorRuntime())
    return np.mean(sample)

def timeSampleFunMean():
    sample = [] 
    for i in range(1000):
        sample.append(functionRuntime())
    return np.mean(sample)

print(f"""Mean runtime for generator is {timeSampleGenMean()}
Mean runtime for function is {timeSampleFunMean()}""")


Mean runtime for generator is 4.399728775024414e-05
Mean runtime for function is 4.000091552734375e-05


In [292]:
# Well it seems that it doesn't really for a reason, the function is stil faster
# I ran the cell multiple times and the mayority was always lower runtime the function

# However fileLines() is a generator of lines from a file whose output (the lines) are not
# stored in memory, so if we tried to call the generator we would just create one instance of it

lineGenerator = fileLines("Hola.txt")

for i in range(8):
    print(next(lineGenerator))

# The generator is something that can be stored in memory, but the results are not

Hola

Adios

Como vamos

Bien y vos

Estoy concentrado

Yo tambien

Vamos ganando

Ya lo sÃ©


In [305]:
lineGenerator == print(lineGenerator)

<generator object fileLines at 0x000001F37F2F65C0>


False

In [306]:
# The code above only gave us the possible number of cars sold in a week
# in one week, however, we can simulate (with more code), the entire store

def car_generator(env):
    # Generate cars from private individuals according to Poisson process with lambda = 5
    while True:
        yield env.timeout(np.random.poisson(5)) # Wait for next car arrival 
        print(f"A car arrived at {env.now}")

def customer_generator(env):
    # Generate customers according to Poisson process with lambda = 20
    while True:
        yield env.timeout(np.random.poisson(20)) # Wait for next customer arrival
        print(f"A customer arrived at {env.now}")

def car_store(env):
    # Simulate the interaction between cars and customers
    while True:
        car_event = env.process(car_generator(env)) # Start generating cars
        customer_event = env.process(customer_generator(env)) # Start generating customers
        
        yield simpy.AnyOf(env, [car_event, customer_event]) # Wait for either event to occur
        
        if car_event.triggered: # If a car arrived
            print("A car is available for sale")
            
            if len(customer_event.callbacks) > 0: # If there are waiting customers
                print("A customer buys the car")
                customer_event.callbacks.pop(0).succeed() # Remove one customer from queue
                
            else: # If there are no waiting customers
                print("The car remains unsold")
                
        if customer_event.triggered: # If a customer arrived
            print("A customer is looking for a car")
            
            if len(car_event.callbacks) > 0: # If there are available cars
                print("The customer finds a car")
                car_event.callbacks.pop(0).succeed() # Remove one car from queue
                
            else: # If there are no available cars 
                print("The customer leaves without buying")

env = simpy.Environment() # Create an environment object

store = env.process(car_store(env)) # Start simulating the store

env.run(until=100) # Run the simulation until time 100

car_store(env)


A car arrived at 7
A car arrived at 11
A car arrived at 16
A car arrived at 21
A car arrived at 26
A customer arrived at 27
A car arrived at 32
A car arrived at 39
A car arrived at 45
A customer arrived at 48
A car arrived at 51
A car arrived at 57
A customer arrived at 61
A car arrived at 65
A car arrived at 68
A car arrived at 72
A customer arrived at 74
A car arrived at 74
A car arrived at 83
A car arrived at 88
A car arrived at 92
A customer arrived at 98
A car arrived at 99


<generator object car_store at 0x000001F37F2ABE20>

In [327]:
# We can also use an exponential distribution to simulate this 
# in a continuous way

# Define parameters
CAR_ARRIVAL_RATE = 5 # cars per week
CUSTOMER_ARRIVAL_RATE = 20 # customers per week
SIM_TIME = 52 # weeks
SELL_PROBABILITY = 0.1 # probability of selling a car

# Create simulation environment
env = simpy.Environment()

# Define processes
def receive_car(env):
    global inventory
    """A process that generates cars at an exponential interarrival time."""
    while True:
        yield env.timeout(np.random.exponential(1/CAR_ARRIVAL_RATE))
        print(f"Received a car at {env.now:.2f} weeks.")
        inventory += 1

def sell_car(env):
    """A process that generates customers at an exponential interarrival time and sells cars with a given probability if available."""
    global inventory # number of cars in stock
    global sells
    sells = 0
    inventory = 0 
    while True:
        yield env.timeout(np.random.exponential(1/CUSTOMER_ARRIVAL_RATE))
        print(f"A customer arrived at {env.now:.2f} weeks.")
        print(f"There are {inventory} cars available.")
        if inventory > 0:
            outcome = np.random.choice([True, False], p=[SELL_PROBABILITY, 1-SELL_PROBABILITY])
            if outcome == True:
                inventory -= 1
                sells += 1
                print(f"Sold a car at {env.now:.2f} weeks.")
            else:
                print(f"Did not sell a car at {env.now:.2f} weeks.")
        else:
            print(f"No car available at {env.now:.2f} weeks.")

# Create processes and run simulation
env.process(receive_car(env))
env.process(sell_car(env))
env.run(until=SIM_TIME)

# I am just adding some things here to experiment, it is not necessary to do this fot the simulation
# per se, but if we are talking about a real store, it would be useful to know hoe much do we sell
print(f"""
There were {sells} sells, and a total of ${sells * 12000} received
Accounting for a costs of reparation, employess, etc. The profit would be {(sells * 12000) - 100000}
""")

# We see that even with 0.1 probability of selling we do in fact make a lot of money
# OF course, this is hyothetical and maybe a horribile approximation, but we would the hell sell!

Received a car at 0.04 weeks.
A customer arrived at 0.04 weeks.
There are 1 cars available.
Did not sell a car at 0.04 weeks.
Received a car at 0.10 weeks.
Received a car at 0.14 weeks.
A customer arrived at 0.15 weeks.
There are 3 cars available.
Did not sell a car at 0.15 weeks.
A customer arrived at 0.17 weeks.
There are 3 cars available.
Did not sell a car at 0.17 weeks.
Received a car at 0.18 weeks.
A customer arrived at 0.18 weeks.
There are 4 cars available.
Did not sell a car at 0.18 weeks.
A customer arrived at 0.24 weeks.
There are 4 cars available.
Did not sell a car at 0.24 weeks.
Received a car at 0.29 weeks.
A customer arrived at 0.31 weeks.
There are 5 cars available.
Did not sell a car at 0.31 weeks.
A customer arrived at 0.33 weeks.
There are 5 cars available.
Did not sell a car at 0.33 weeks.
Received a car at 0.36 weeks.
A customer arrived at 0.37 weeks.
There are 6 cars available.
Did not sell a car at 0.37 weeks.
A customer arrived at 0.37 weeks.
There are 6 cars a

In [330]:
# Skipping to DES
import simpy
import numpy as np

env = simpy.Environment()

# Define the resources
machine = simpy.Resource(env, capacity=3)

# Define the processes
def manufacturing_process(env, machine):
    # Wait for a machine to become available
    with machine.request() as request:
        # Wait for the machine to become available
        yield request
        # Wait for the manufacturing process to complete
        yield env.timeout(np.random.uniform(0, 1))

# Run the simulation
env.process(manufacturing_process(env, machine))
env.run(until=1)
